In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
#from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle

In [26]:
# Assuming your data is in a CSV file named 'your_data.csv'
data = pd.read_csv('Robot_train_4s.csv')

# 1. Data Preprocessing
# Example: Handle missing values
data = data.dropna()

In [3]:
data

,Label,X,Y,Z,X*Y,X*Z,Y*Z,X*Y+Z,X*Y*Z,X*Z+Y,...,slop(1/z),slop(1/x),slop(z/x),slop(z),slop(x)^2,slop(x/z)^2,slop(1/z)^2,slop(1/x)^2,slop(z/x)^2,slop(z)^2
0,-1,1044.181085,0.746138,0.387314,779.103186,404.425953,404.425953,779.490500,301.757571,405.172091,...,91.199270,0.62300,0.006832,0.010965,2.575925,21424.762920,8320.000000,0.388000,0.000047,0.000120
1,-1,1042.576116,0.746192,0.376349,777.961957,392.372479,392.372479,778.338306,292.785205,393.118671,...,153.092468,0.62300,0.004067,0.006532,2.579364,60453.348740,23400.000000,0.388000,0.000017,0.000043
2,-1,1040.970076,0.746208,0.369817,776.780198,384.968431,384.968431,777.150015,287.266523,385.714639,...,194.628260,0.62300,0.003198,0.005138,2.580556,97751.884530,37900.000000,0.388000,0.000010,0.000026
3,-1,1039.363665,0.746214,0.364679,775.587718,379.034102,379.034102,775.952397,282.840553,379.780316,...,212.856535,0.62200,0.002924,0.004698,2.580942,116937.066700,45300.000000,0.387000,0.000009,0.000022
4,-1,1037.757134,0.746215,0.359981,774.389940,373.572851,373.572851,774.749921,278.765665,374.319066,...,219.394471,0.62200,0.002837,0.004558,2.581067,124236.916600,48100.000000,0.387000,0.000008,0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145678,16&17,-484.315636,0.696923,-121.823592,-337.530706,59001.070440,-84.901663,-459.354298,41119.203010,59001.767360,...,7.697518,0.61500,0.079893,0.129912,2.644100,156.667677,59.300000,0.378000,0.006380,0.016877
145679,16&17,-485.941705,0.696923,-121.953504,-338.663951,59262.293660,-84.992202,-460.617455,41301.255490,59262.990590,...,7.697459,0.61500,0.079894,0.129913,2.644097,156.665072,59.300000,0.378000,0.006380,0.016877
145680,16&17,-487.567773,0.696923,-122.083417,-339.797195,59523.939750,-85.082741,-461.880612,41483.602660,59524.636670,...,7.697518,0.61500,0.079893,0.129912,2.644100,156.667677,59.300000,0.378000,0.006380,0.016877
145681,16&17,-489.193842,0.696923,-122.213329,-340.930440,59786.007960,-85.173280,-463.143769,41666.244020,59786.704880,...,7.697459,0.61500,0.079894,0.129913,2.644100,156.665265,59.300000,0.378000,0.006380,0.016877


In [27]:
y = data.pop('Label')
x = data
print(x.shape,y.shape)

(145683, 38) (145683,)


In [5]:
print(data.keys())

Index(['X', 'Y', 'Z', 'X*Y', 'X*Z', 'Y*Z', 'X*Y+Z', 'X*Y*Z', 'X*Z+Y', 'Y*Z+X',
       'X+Y', 'X+Z', 'Y+Z', 'X+Y+Z', 'X-Y', 'X-Z', 'Y-Z', 'X+Y-Z', 'X-Y+Z',
       'X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)',
       'slop(x)^2', 'slop(x/z)^2', 'slop(1/z)^2', 'slop(1/x)^2', 'slop(z/x)^2',
       'slop(z)^2'],
      dtype='object')


In [28]:
# Convert all columns to integers
x = x.astype(int)
x

c:\Users\Sameh\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


,X,Y,Z,X*Y,X*Z,Y*Z,X*Y+Z,X*Y*Z,X*Z+Y,Y*Z+X,...,slop(1/z),slop(1/x),slop(z/x),slop(z),slop(x)^2,slop(x/z)^2,slop(1/z)^2,slop(1/x)^2,slop(z/x)^2,slop(z)^2
0,1044,0,0,779,404,404,779,301,405,1044,...,91,0,0,0,2,21424,8320,0,0,0
1,1042,0,0,777,392,392,778,292,393,1042,...,153,0,0,0,2,60453,23400,0,0,0
2,1040,0,0,776,384,384,777,287,385,1041,...,194,0,0,0,2,97751,37900,0,0,0
3,1039,0,0,775,379,379,775,282,379,1039,...,212,0,0,0,2,116937,45300,0,0,0
4,1037,0,0,774,373,373,774,278,374,1038,...,219,0,0,0,2,124236,48100,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145678,-484,0,-121,-337,59001,-84,-459,41119,59001,-569,...,7,0,0,0,2,156,59,0,0,0
145679,-485,0,-121,-338,59262,-84,-460,41301,59262,-570,...,7,0,0,0,2,156,59,0,0,0
145680,-487,0,-122,-339,59523,-85,-461,41483,59524,-572,...,7,0,0,0,2,156,59,0,0,0
145681,-489,0,-122,-340,59786,-85,-463,41666,59786,-574,...,7,0,0,0,2,156,59,0,0,0


In [7]:
#x =  x[['X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [8]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=12,n_estimators=4)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.9110409445035522
Memory usage of Random Forest model: 5.003867  M bytes


In [9]:
# 2. Splitting the Data
x =  x[['X', 'Y', 'Z', 'X*Y', 'X*Z', 'Y*Z', 'X*Y+Z', 'X*Y*Z', 'X*Z+Y', 'Y*Z+X',
       'X+Y', 'X+Z', 'Y+Z', 'X+Y+Z', 'X-Y', 'X-Z', 'Y-Z', 'X+Y-Z', 'X-Y+Z',
       'X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)']]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [10]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=13,n_estimators=20)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'X', 'Z', 'X*Z', 'X+Z', 'X-Z', 'X/Z', 'Z/X'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.7433160586196246
Memory usage of Random Forest model: 48.388224  M bytes


In [15]:
x =  x[['X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)',
       'slop(x)^2', 'slop(x/z)^2', 'slop(1/z)^2', 'slop(1/x)^2', 'slop(z/x)^2']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [16]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=13,n_estimators=6)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'
Accuracy :  0.9086041802519134
Memory usage of Random Forest model: 10.784818  M bytes


In [21]:
x =  x[['X+Y', 'X+Z', 'Y+Z', 'X+Y+Z', 'X-Y', 'X-Z', 'Y-Z', 'X+Y-Z', 'X-Y+Z',
       'X-Y-Z', 'X/Y', 'X/Z', 'Y/Z', 'Y/X', 'Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [22]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=10,n_estimators=8)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.7243024333321894
Memory usage of Random Forest model: 7.284588  M bytes


In [23]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=10,n_estimators=6)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.6691491917493222
Memory usage of Random Forest model: 5.712666  M bytes


In [24]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=10,n_estimators=5)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.6427566324604455
Memory usage of Random Forest model: 4.852001  M bytes


In [29]:
x =  x[['Z/X', 'Z/Y', 'slop(x)',
       'slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)',
       'slop(x)^2', 'slop(x/z)^2', 'slop(1/z)^2', 'slop(1/x)^2']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [30]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=30,n_estimators=10)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("7 Features : 'Z/X', 'slop(x)','slop(x/z)', 'slop(1/z)', 'slop(1/x)', 'slop(z/x)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.8687236160208669
Memory usage of Random Forest model: 181.649394  M bytes


float dataset


In [26]:
#x =  x[['X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [29]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=14,n_estimators=4)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.9725091807667228
Memory usage of Random Forest model: 4.496386  M bytes


In [30]:
#x =  x[['X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)']]
X_train,X_test,y_train,y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [31]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=13,n_estimators=4)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.9633112537323678
Memory usage of Random Forest model: 3.83693  M bytes


In [34]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=11,n_estimators=4)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.8923362048254796
Memory usage of Random Forest model: 2.348002  M bytes


In [35]:
rf_model_1 = RandomForestClassifier(random_state=42,max_depth=12,n_estimators=4)
rf_model_1.fit(X_train, y_train)

# 4. Model Evaluation
y_pred_1 = rf_model_1.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred_1)

#print("5 Features : 'X', 'Z', 'slop(x)', 'slop(x/z)', 'slop(z)'")

#print(classification_report(y_test,y_pred_5))
print('Accuracy : ', accuracy_score(y_test, y_pred_1))

# Serialize the model to a bytes object
model_bytes = pickle.dumps(rf_model_1)

# Get the memory usage of the serialized model
memory_usage = sys.getsizeof(model_bytes)
print("Memory usage of Random Forest model:", memory_usage/1000000, " M bytes")

Accuracy :  0.9401448330301678
Memory usage of Random Forest model: 2.84517  M bytes
